```
Walmart Case Study
Apriory Alogritham
Market Basket Analysis
Association Rules
Association Metrics
```

```
Market Basket Analysis : Identify items combinations that occure together very Frequently
```

# Importing Libraries

In [44]:
import warnings;warnings.filterwarnings('ignore')
import os,sys,joblib,math,json
from datetime import date,time,datetime,timedelta
from tqdm.auto import tqdm;tqdm.pandas()
import numpy as np,pandas as pd
pd.set_option('display.max_columns',None)


from mlxtend.frequent_patterns import apriori,association_rules

# Load Data & PostProcessing (Walmart Case Study)

```
one basket --> one Invoice --> one transaction

-ve Quantity means Return Items, we need to remove them
Remove rows where InvoiceNO is Null
Romve rows where Invoice contains 'C', that is acutally the invoice

Description is the Product Name
```

In [28]:
df = pd.read_csv('data/Online_Retail.csv')
print(df.shape)
df.head(2)

(541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/10 8:26,3.39,17850.0,United Kingdom


In [29]:
# Remove Noice
df = df[df.Quantity > 0]                   # Remove returned items
df = df.dropna(subset=['InvoiceNo'])
df = df[~df.InvoiceNo.str.contains('C')]   # Remove all the transaction done by credit card

# Only Take United Kingdom Data
df = df[df.Country == 'United Kingdom']

In [ ]:
print(f"No of Customers we have : {df.CustomerID.nunique()}")
print(f"No of Invoices we have: {df.InvoiceNo.nunique()}")

No of Customers we have : 3921
No of Invoices we have: 18786


In [31]:
# Create Bucket Item Matrics (Basket Item Occurance Metrics)
df_bucket_item = df.groupby(['InvoiceNo','Description'])['Quantity'].sum().unstack().fillna(0)
print(df_bucket_item.shape)
df_bucket_item.head()

# There are total 18194 baskets and total unique items are 4058

(18194, 4058)


Description   4 PURPLE FLOCK DINNER CANDLES   50'S CHRISTMAS GIFT BAG LARGE  \
InvoiceNo                                                                     
536365                                  0.0                             0.0   
536366                                  0.0                             0.0   
536367                                  0.0                             0.0   
536368                                  0.0                             0.0   
536369                                  0.0                             0.0   

Description   DOLLY GIRL BEAKER   I LOVE LONDON MINI BACKPACK  \
InvoiceNo                                                       
536365                      0.0                           0.0   
536366                      0.0                           0.0   
536367                      0.0                           0.0   
536368                      0.0                           0.0   
536369                      0.0                           0.0   

Description   NINE DRAWER OFFICE TIDY   OVAL WALL MIRROR DIAMANTE   \
InvoiceNo                                                            
536365                            0.0                          0.0   
536366                            0.0                          0.0   
536367                            0.0                          0.0   
536368                            0.0                          0.0   
536369                            0.0                          0.0   

Description   RED SPOT GIFT BAG LARGE   SET 2 TEA TOWELS I LOVE LONDON   \
InvoiceNo                                                                 
536365                            0.0                               0.0   
536366                            0.0                               0.0   
536367                            0.0                               0.0   
536368                            0.0                               0.0   
536369                            0.0                               0.0   

Description   SPACEBOY BABY GIFT SET   TOADSTOOL BEDSIDE LIGHT   \
InvoiceNo                                                         
536365                           0.0                        0.0   
536366                           0.0                        0.0   
536367                           0.0                        0.0   
536368                           0.0                        0.0   
536369                           0.0                        0.0   

Description   TRELLIS COAT RACK  *Boombox Ipod Classic  \
InvoiceNo                                                
536365                      0.0                    0.0   
536366                      0.0                    0.0   
536367                      0.0                    0.0   
536368                      0.0                    0.0   
536369                      0.0                    0.0   

Description  *USB Office Mirror Ball  10 COLOUR SPACEBOY PEN  \
InvoiceNo                                                      
536365                           0.0                     0.0   
536366                           0.0                     0.0   
536367                           0.0                     0.0   
536368                           0.0                     0.0   
536369                           0.0                     0.0   

Description  12 COLOURED PARTY BALLOONS  12 DAISY PEGS IN WOOD BOX  \
InvoiceNo                                                            
536365                              0.0                        0.0   
536366                              0.0                        0.0   
536367                              0.0                        0.0   
536368                              0.0                        0.0   
536369                              0.0                        0.0   

Description  12 EGG HOUSE PAINTED WOOD  12 HANGING EGGS HAND PAINTED  \
InvoiceNo                                                              
536365      

In [32]:
# Make Data Binary
df_bucket_item = df_bucket_item.applymap(lambda x: 0 if x == 0 else 1)

In [ ]:
# Take only thoes Invoices in which we have atleast 2 item
df_bucket_item = df_bucket_item[df_bucket_item.sum(axis=1)>=2]
df_bucket_item.shape

# Apriory Algorithaham

```
if there are n items then we can create 2^n subsets (Item Combination)
Most Freqent combination will be recommended that is the Apriory Alogritham


Drawback : if n is large, 2^n is very large

Now to reduce complexity, idea, is lets say if frequency of item {a} in baskets is 20 then the frequency of {a,b} will be less the {a} in the baskets
so if item a occured x no of times in out of n baskets then any combination of item a with other item will have frequency less than or equal to x
creating hurestic rule if frequency of any combination is less than threshold then we will stop including that conbination



Support : (Occurence of the item combination in baskets) / (total number of baskets) [0,1]


```

In [37]:
df_bucket_item.head(3)

Description   4 PURPLE FLOCK DINNER CANDLES   50'S CHRISTMAS GIFT BAG LARGE  \
InvoiceNo                                                                     
536365                                    0                               0   
536366                                    0                               0   
536367                                    0                               0   

Description   DOLLY GIRL BEAKER   I LOVE LONDON MINI BACKPACK  \
InvoiceNo                                                       
536365                        0                             0   
536366                        0                             0   
536367                        0                             0   

Description   NINE DRAWER OFFICE TIDY   OVAL WALL MIRROR DIAMANTE   \
InvoiceNo                                                            
536365                              0                            0   
536366                              0                            0   
536367                              0                            0   

Description   RED SPOT GIFT BAG LARGE   SET 2 TEA TOWELS I LOVE LONDON   \
InvoiceNo                                                                 
536365                              0                                 0   
536366                              0                                 0   
536367                              0                                 0   

Description   SPACEBOY BABY GIFT SET   TOADSTOOL BEDSIDE LIGHT   \
InvoiceNo                                                         
536365                             0                          0   
536366                             0                          0   
536367                             0                          0   

Description   TRELLIS COAT RACK  *Boombox Ipod Classic  \
InvoiceNo                                                
536365                        0                      0   
536366                        0                      0   
536367                        0                      0   

Description  *USB Office Mirror Ball  10 COLOUR SPACEBOY PEN  \
InvoiceNo                                                      
536365                             0                       0   
536366                             0                       0   
536367                             0                       0   

Description  12 COLOURED PARTY BALLOONS  12 DAISY PEGS IN WOOD BOX  \
InvoiceNo                                                            
536365                                0                          0   
536366                                0                          0   
536367                                0                          0   

Description  12 EGG HOUSE PAINTED WOOD  12 HANGING EGGS HAND PAINTED  \
InvoiceNo                                                              
536365                               0                             0   
536366                               0                             0   
536367                               0                             0   

Description  12 IVORY ROSE PEG PLACE SETTINGS  \
InvoiceNo                                       
536365                                      0   
536366                                      0   
536367                                      0   

Description  12 MESSAGE CARDS WITH ENVELOPES  12 PENCIL SMALL TUBE WOODLAND  \
InvoiceNo                                                                     
536365                                     0                              0   
536366                                     0                              0   
536367                                     0                              0   

Description  12 PENCILS SMALL TUBE RED RETROSPOT  12 PENCILS SMALL TUBE SKULL  \
InvoiceNo                                                                       
536365                                         0                            0   
536366

In [41]:
most_items_occured_togeter = apriori(df_bucket_item,min_support=0.03,use_colnames=True).sort_values('support',ascending=False).reset_index(drop=True)
most_items_occured_togeter['len_of_combination'] = most_items_occured_togeter.itemsets.agg(len)

In [43]:
most_items_occured_togeter

,support,itemsets,len_of_combination
0,0.129875,(WHITE HANGING HEART T-LIGHT HOLDER),1
1,0.116331,(JUMBO BAG RED RETROSPOT),1
2,0.100671,(REGENCY CAKESTAND 3 TIER),1
3,0.095471,(PARTY BUNTING),1
4,0.084165,(LUNCH BAG RED RETROSPOT),1
...,...,...,...
172,0.030473,(CHRISTMAS CRAFT LITTLE FRIENDS),1
173,0.030473,(CREAM HEART CARD HOLDER),1
174,0.030413,"(LUNCH BAG BLACK SKULL., LUNCH BAG CARS BLUE)",2
175,0.030292,"(LUNCH BAG SPACEBOY DESIGN , LUNCH BAG RED RET...",2


```
Association Rules
Milk + Break
Bread + Butter
Pizza + Coke  P(Coke/Pizza) is High 
Pizza : Anticedent
Coke : Consequent



Note : P(Coke/Pizza) != P(Pizza/Coke)


Association Metrics:
1) CONFIDENCE
Confidence(Coke/Pizza):  support(Coke,Pizza)/ support(Pizza) : (no of buckets having Coke and pizza together) / (no of buckets having pizza only)



2)LIFT
Lift(Coke) : support(x,y)/ support(x)* support(y)   [0,inf]
lift(X,Y) = 1  : if x and y have no correlation then support(x,y) = support(x) * support(y)
          > 1  : if x and y have positive correlation
          < 1  : if x and y have negative correlation


3)CONVICTION (same as lift) [-1,1]
conviction : support(x,y) -  support(x)* support(y)
conviction(x,y) = 0 : no correlation
           > 0 : possitiove correlation between x and y
           < 0 : negative correlation between x and y
```

In [47]:
associtaion_rules_df = association_rules(most_items_occured_togeter,metric='lift',min_threshold=1).sort_values('lift',ascending=False).reset_index(drop=True)

In [48]:
associtaion_rules_df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.042264,0.056473,0.034887,0.825465,14.617093,1.0,0.032500,5.405948,0.972697,0.546402,0.815019,0.721619
1,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.056473,0.042264,0.034887,0.617773,14.617093,1.0,0.032500,2.505674,0.987345,0.546402,0.600906,0.721619
2,(PINK REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.042264,0.057682,0.033013,0.781116,13.541798,1.0,0.030575,4.305101,0.967025,0.493225,0.767717,0.676721
3,(ROSES REGENCY TEACUP AND SAUCER ),(PINK REGENCY TEACUP AND SAUCER),0.057682,0.042264,0.033013,0.572327,13.541798,1.0,0.030575,2.239413,0.982847,0.493225,0.553454,0.676721
4,(GARDENERS KNEELING PAD CUP OF TEA ),(GARDENERS KNEELING PAD KEEP CALM ),0.045287,0.054235,0.032711,0.722296,13.317793,1.0,0.030254,3.405662,0.968786,0.489593,0.706371,0.662709
5,(GARDENERS KNEELING PAD KEEP CALM ),(GARDENERS KNEELING PAD CUP OF TEA ),0.054235,0.045287,0.032711,0.603122,13.317793,1.0,0.030254,2.405555,0.977952,0.489593,0.584296,0.662709
6,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.056473,0.057682,0.042385,0.750535,13.011639,1.0,0.039127,3.777361,0.978398,0.590564,0.735265,0.742668
7,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.057682,0.056473,0.042385,0.734801,13.011639,1.0,0.039127,3.557807,0.979654,0.590564,0.718928,0.742668
8,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.056170,0.052663,0.034041,0.606028,11.507574,1.0,0.031083,2.404578,0.967442,0.455133,0.584127,0.626206
9,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.052663,0.056170,0.034041,0.646383,11.507574,1.0,0.031083,2.669077,0.963861,0.455133,0.625339,0.626206
